# LETS GET STARTED WITH DATA COLLECTION
Now we have two options either we create synthetic data or look for a  real one online 
After careful research through various datsets online on differnt platforms we found Olist dataset on kaggle 
why this ?
1.)The entire Olist dataset represents Brazilian e-commerce transactions collected from a large marketplace platform.

Olist is a real Brazilian e-commerce company headquartered in Curitiba, Brazil.

2.)The dataset was officially released on Kaggle in 2018, but the underlying transactions were collected earlier:

Transaction period: 2016 – 2018

(verified from order_purchase_timestamp in olist_orders_dataset.csv)

So:

The data is real

Collected over 2 years

From hundreds of thousands of Brazilian shoppers

Across multiple cities and ZIP codes

✅ Who released it?

The dataset was released by Olist, through their data science team, as part of a public research initiative.


In [5]:
import pandas as pd

# file paths
CUSTOMERS_CSV = "olist_customers_dataset.csv"
ORDERS_CSV = "olist_orders_dataset.csv"
ORDER_ITEMS_CSV = "olist_order_items_dataset.csv"
PAYMENTS_CSV = "olist_order_payments_dataset.csv"
REVIEWS_CSV = "olist_order_reviews_dataset.csv"
PRODUCTS_CSV = "olist_products_dataset.csv"

OUT_CSV = "final_data.csv"

# load data
orders = pd.read_csv(ORDERS_CSV, parse_dates=[
    "order_purchase_timestamp",
    "order_approved_at",
    "order_delivered_carrier_date",
    "order_delivered_customer_date"
], low_memory=False)

customers = pd.read_csv(CUSTOMERS_CSV, low_memory=False)
order_items = pd.read_csv(ORDER_ITEMS_CSV, low_memory=False)
payments = pd.read_csv(PAYMENTS_CSV, low_memory=False)
reviews = pd.read_csv(REVIEWS_CSV, parse_dates=[
    "review_creation_date",
    "review_answer_timestamp"
], low_memory=False)
products = pd.read_csv(PRODUCTS_CSV, low_memory=False)

# Step 1: orders ← customers
df = orders.merge(customers, on="customer_id", how="left")

# Step 2: add order_items
df = df.merge(order_items, on="order_id", how="left")

# Step 3: add product metadata
df = df.merge(products, on="product_id", how="left")

# Step 4: aggregate payments
payments_agg = payments.groupby("order_id").agg({
    "payment_value": "sum",
    "payment_type": lambda x: "|".join(sorted(x.unique())),
    "payment_installments": "max"
}).reset_index().rename(columns={
    "payment_value": "payment_value_sum",
    "payment_type": "payment_types",
    "payment_installments": "payment_installments_max"
})

df = df.merge(payments_agg, on="order_id", how="left")

# Step 5: aggregate reviews (latest review per order)
reviews_agg = (
    reviews.sort_values("review_creation_date")
           .groupby("order_id")
           .tail(1)[["order_id", "review_score", "review_comment_message"]]
)

df = df.merge(reviews_agg, on="order_id", how="left")

# clean numeric columns
num_cols = [
    "price","freight_value","payment_value_sum",
    "product_weight_g","product_length_cm",
    "product_height_cm","product_width_cm"
]

for c in num_cols:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0.0)

# final output
print("Final merged shape:", df.shape)
print(df.head())

# save
df.to_csv(OUT_CSV, index=False)
print("Saved:", OUT_CSV)


Final merged shape: (113425, 31)
                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp   order_approved_at  \
0    delivered      2017-10-02 10:56:33 2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37 2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49 2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06 2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39 2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26

In [6]:
data = pd.read_csv('final_data.csv')
data.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,...,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,payment_value_sum,payment_types,payment_installments_max,review_score,review_comment_message
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,7c396fd4830fd04220f754e42b4e5bff,3149,...,4.0,500.0,19.0,8.0,13.0,38.71,credit_card|voucher,1.0,4.0,"Não testei o produto ainda, mas ele veio corre..."
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,af07308b275d755c9edb36a90c618231,47813,...,1.0,400.0,19.0,13.0,19.0,141.46,boleto,1.0,4.0,Muito bom o produto.
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,3a653a41f6f9fc3d2a113cf8398680e8,75265,...,1.0,420.0,24.0,19.0,21.0,179.12,credit_card,3.0,5.0,NaN
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,7c142cf63193a1473d2e66489a9ae977,59296,...,3.0,450.0,30.0,10.0,20.0,72.20,credit_card,1.0,5.0,O produto foi exatamente o que eu esperava e e...
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,72632f0f9dd73dfee390c9b22eb56dd6,9195,...,4.0,250.0,51.0,15.0,15.0,28.62,credit_card,1.0,5.0,NaN
